In [2]:
# 前半部分代码（保持不动，已初始化browser并访问目标页）
%pip install selenium
%pip install webdriver_manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import random
import json
import re  # 补充正则模块（用于提取评论数/星级）
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (TimeoutException, NoSuchElementException,
                                        ElementClickInterceptedException)
from fake_useragent import UserAgent  # 补充UA模块（反爬用）

# 前半部分：启动浏览器+访问目标页+滚动
# webdriverpath = "/Users/ziqizhou/.wdm/drivers/chromedriver/mac64/130.0.6723.69/chromedriver-mac-arm64/chromedriver"
# server = Service(executable_path=webdriverpath)
# browser = webdriver.Chrome(service=server)
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# 补充反爬配置（隐藏webdriver特征，前半部分原有逻辑补充）
browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        Object.defineProperty(navigator, 'languages', {get: () => ['zh-CN', 'zh']});
        Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3]});
    """
})
browser.maximize_window()
# 访问目标URL（宠物殡葬搜索页）
browser.get("https://www.dianping.com/search/keyword/4/0_宠物殡葬")

# 前半部分：滚动逻辑（保持不动）
# 先滚动到页尾
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# 精细化滚动（3秒内小步滚动）
scroll_height = 10  # 每次滚动像素
scroll_delay = 0.05  # 滚动间隔
scroll_duration = 5  # 总滚动时长
end_time = time.time() + scroll_duration
while time.time() < end_time:
    browser.execute_script(f"window.scrollBy(0, {scroll_height});")
    time.sleep(scroll_delay)




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ---------------------- 2. 初始化变量（新增失败计数）----------------------
result = []
target_pages = 16
current_page = 1
output_file = f"pet_funeral_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
empty_page_count = 0  # 连续空页计数（超过3次终止）
max_empty_pages = 3   # 最大允许连续空页数


# 3. 核心函数：提取单页店铺信息（强化空数据判断）
def get_current_page_shops():
    global empty_page_count
    try:
        # 等待店铺列表容器（更严格的验证）
        print(f"⌛ 等待第{current_page}页店铺列表...")
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#shop-all-list"))
        )
    except TimeoutException:
        print("⚠️ 验证码或加载失败！处理后按回车...")
        input()
        try:
            WebDriverWait(browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#shop-all-list"))
            )
        except:
            print(f"❌ 第{current_page}页加载失败")
            empty_page_count += 1
            return []
    
    # 强化滚动（确保懒加载）
    for _ in range(3):
        browser.execute_script(f"window.scrollBy(0, {random.randint(300, 600)});")
        time.sleep(random.uniform(1.5, 2.5))
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(3.0, 4.0))
    
    # 提取店铺容器（关键：验证选择器有效性）
    shop_items = browser.find_elements(By.CSS_SELECTOR, "#shop-all-list > ul > li > div.txt")
    print(f"📊 第{current_page}页找到{len(shop_items)}家店铺")
    
    # 若店铺数为0，递增空页计数
    if len(shop_items) == 0:
        empty_page_count += 1
        print(f"⚠️ 连续空页数：{empty_page_count}/{max_empty_pages}")
        return []
    else:
        empty_page_count = 0  # 有数据则重置空页计数
    
    page_shops = []
    for idx, item in enumerate(shop_items, 1):
        try:
            # 字段提取（每个选择器单独验证，避免整体失败）
            name = item.find_element(By.CSS_SELECTOR, ".tit > a > h4").text.strip() if item.find_elements(By.CSS_SELECTOR, ".tit > a > h4") else f"无名_{idx}"
            link = item.find_element(By.CSS_SELECTOR, "div.tit > a").get_attribute("href") if item.find_elements(By.CSS_SELECTOR, "div.tit > a") else "无链接"
            #score = item.find_element(By.CSS_SELECTOR, ".score .average").text.strip() if item.find_elements(By.CSS_SELECTOR, ".score .average") else "无评分"
            address = item.find_element(By.CSS_SELECTOR, "div.tag-addr > a:nth-child(3) > span").text.strip() if item.find_elements(By.CSS_SELECTOR, "div.tag-addr > a:nth-child(3) > span") else "无地址"
            avg_price = item.find_element(By.CSS_SELECTOR, ".mean-price").text.strip() if item.find_elements(By.CSS_SELECTOR, ".mean-price") else "无人均"
            
            # 评论数（正则提取）
            comment_num = "无评论数"
            comment_elem = item.find_elements(By.CSS_SELECTOR, "div.comment > a.review-num")
            if comment_elem:
                match = re.search(r"\d+", comment_elem[0].text)
                if match:
                    comment_num = match.group()
            
            # 星级（解析star_xx）
            star = "无星级"
            star_container = item.find_elements(By.CSS_SELECTOR, ".nebula_star .star_icon")
            if star_container:
                star_spans = star_container[0].find_elements(By.CSS_SELECTOR, ".star")
                if star_spans:
                    star_class = star_spans[0].get_attribute("class")
                    match = re.search(r"star_(\d+)", star_class)
                    if match:
                        star = f"{int(match.group(1))/10}分"
            
            page_shops.append({
                "page": current_page, "index": idx, "name": name, "link": link,
                "address": address, "avg_price": avg_price,
                "comment_num": comment_num, "star": star
            })
            print(f"  ✅ 第{idx}家：{name[:10]}... | 人均：{avg_price}")
        except Exception as e:
            print(f"  ❌ 第{idx}家失败：{str(e)[:40]}")
            continue
    
    # 若当前页提取的数据为空（所有店铺都失败），递增空页计数
    if not page_shops:
        empty_page_count += 1
        print(f"⚠️ 第{current_page}页无有效数据，连续空页数：{empty_page_count}")
    return page_shops


# 4. 翻页函数（强化状态判断，避免无限翻页）
def go_to_next_page():
    global current_page
    try:
        # 验证下一页按钮存在且可点击
        next_btn = WebDriverWait(browser, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".page a.next"))
        )
        
        # 检查按钮是否禁用（部分网站禁用时会加disabled类）
        if "disabled" in next_btn.get_attribute("class"):
            print("❌ 下一页按钮已禁用（已到最后一页）")
            return False
        
        # 滚动并点击
        browser.execute_script("arguments[0].scrollIntoView(true);", next_btn)
        time.sleep(random.uniform(2.0, 3.0))
        next_btn.click()
        
        # 记录点击前的页码，用于验证翻页是否成功
        prev_page = current_page
        current_page += 1
        
        # 等待新页面加载并验证URL变化（关键：避免假翻页）
        time.sleep(random.uniform(5.0, 7.0))
        current_url = browser.current_url
        if str(current_page) not in current_url:
            print(f"⚠️ 翻页失败（URL未更新），当前页仍为{prev_page}")
            current_page = prev_page  # 回滚页码
            return False
        
        print(f"✅ 成功翻到第{current_page}页")
        return True
    
    except (NoSuchElementException, TimeoutException):
        print(f"❌ 无下一页（当前最大页：{current_page}）")
        return False
    except ElementClickInterceptedException:
        print("⚠️ 翻页按钮被遮挡，处理后按回车...")
        input()
        return go_to_next_page()


# 5. 主流程（增加强制终止条件）
def main():
    global current_page, result, empty_page_count
    try:
        print("\n📌 开始爬取宠物殡葬店铺数据")
        print(f"当前URL：{browser.current_url}")
        input("确认页面正常后按回车开始...")
        
        while current_page <= target_pages:
            # 强制终止条件1：连续空页超过阈值
            if empty_page_count >= max_empty_pages:
                print(f"❌ 连续{max_empty_pages}页无数据，强制终止")
                break
            
            print(f"\n===== 爬取第{current_page}/{target_pages}页 =====")
            current_shops = get_current_page_shops()
            
            # 强制终止条件2：当前页无数据且已到目标页附近
            if not current_shops and current_page > target_pages * 0.8:
                print(f"❌ 接近目标页仍无数据，终止爬取")
                break
            
            if current_shops:
                result.extend(current_shops)
                print(f"累计爬取：{len(result)}家")
            
            # 翻页判断（未到目标页且能翻页才继续）
            if current_page < target_pages:
                if not go_to_next_page():
                    print("⚠️ 无法翻页，终止爬取")
                    break
            else:
                # 已到目标页，无需翻页
                break
        
        # 保存数据
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        print(f"\n🎉 爬取结束，共{len(result)}条数据，保存至：{output_file}")
    
    except Exception as e:
        print(f"\n❌ 主流程异常：{str(e)[:80]}")
        if result:
            with open(f"backup_{output_file}", "w") as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            print(f"⚠️ 已备份{len(result)}条数据")
    
    finally:
        if 'browser' in locals() and browser:
            browser.quit()
        print("📌 浏览器已关闭")


if __name__ == "__main__":
    main()


📌 开始爬取宠物殡葬店铺数据
当前URL：https://www.dianping.com/search/keyword/4/0_%E5%AE%A0%E7%89%A9%E6%AE%A1%E8%91%AC

===== 爬取第1/16页 =====
⌛ 等待第1页店铺列表...
📊 第1页找到15家店铺
  ✅ 第1家：宠云栖·轻奢宠物善终... | 人均：人均 ￥382
  ✅ 第2家：尾云禅室·宠物火化善... | 人均：人均 ￥341
  ✅ 第3家：终沐·宠物火化殡葬善... | 人均：人均 ￥876
  ✅ 第4家：安念宠物纪念馆·宠物... | 人均：人均 ￥479
  ✅ 第5家：终沐·宠物火化殡葬善... | 人均：人均 ￥522
  ✅ 第6家：礼物LIVE宠物·善... | 人均：人均 ￥376
  ✅ 第7家：宠福安堂•宠物火化殡... | 人均：人均 ￥670
  ✅ 第8家：炎安苑·宠物善终·火... | 人均：人均 ￥1040
  ✅ 第9家：宠渡·宠物善终服务·... | 人均：人均 ￥580
  ✅ 第10家：宠福安堂•宠物火化殡... | 人均：人均 ￥579
  ✅ 第11家：炎安苑·宠物善终·火... | 人均：人均 -
  ✅ 第12家：炎安苑·宠物善终·火... | 人均：人均 ￥577
  ✅ 第13家：如尘精致宠物善终... | 人均：人均 ￥881
  ✅ 第14家：鲸落·精致宠物善终火... | 人均：人均 ￥468
  ✅ 第15家：方树回生·宠物善终服... | 人均：人均 ￥1227
累计爬取：15家
✅ 成功翻到第2页

===== 爬取第2/16页 =====
⌛ 等待第2页店铺列表...
📊 第2页找到15家店铺
  ✅ 第1家：安然宠物善终火化殡葬... | 人均：人均 ￥613
  ✅ 第2家：Novapet诺娃宠... | 人均：人均 -
  ✅ 第3家：萌爪星河·宠物善终·... | 人均：人均 ￥254
  ✅ 第4家：宠屿Pet Isle... | 人均：人均 ￥652
  ✅ 第5家：诺娃Novapet宠... | 人均：人均 ￥984
  ✅ 第6家：广州安壹宠物火化殡葬... | 人均：人均 ￥711
  ✅ 第7家：礼物LIVE宠物·善... | 人均：人均 ￥677
  ✅ 第8家：

In [4]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import json

# 1. 读取JSON文件（你的爬取结果文件）
json_file_path = "/Users/syx/Documents/python脚本/代码/大众点评爬取代码/pet_funeral_20251207_232426.json"  # 替换为你的JSON文件路径
with open(json_file_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)  # 读取JSON数据（列表格式，对应多个店铺）

# 2. 转换为Excel
excel_save_path = "pet_funeral_guangzhou.xlsx"  # 转换后Excel的保存路径
# 用pandas将列表格式的JSON数据转为DataFrame，再写入Excel
df = pd.DataFrame(json_data)
df.to_excel(excel_save_path, index=False, engine="openpyxl")  # index=False：不保留行号

print(f"✅ JSON转换Excel完成！文件已保存至：{excel_save_path}")

✅ JSON转换Excel完成！文件已保存至：pet_funeral_guangzhou.xlsx
